In [ ]:
# Detecting suspicious communication specificly (cyber bulling, threatning, terirosim ) from any type of txt data

### TO-DO
----------
Modello:
- k-Cross-validation
- ridistribuire-dati (10 sospetto 10 non-sospetto)
  - questo devo lavorare sui dati dal di fuori con uno script python, cercare di prendere dati 50/50
- provare-random-under-sampler
- ~~serializzare il modello - pickle~~

Estensione:
- salvare anche i dati che leggo
- in modo da costruire statistiche


#### Lib
----------
**NLTK** is a leading platform for building Python programs to work with human language data. The Natural Language Toolkit (NLTK) is an open source Python library for Natural Language Processing.

**sklearn** is a free software machine learning library for the Python programming language, it features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN.

**Seaborn** is a Python data visualization library based on matplotlib.

**pandas** 

**numpy**

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
from nltk.corpus import stopwords
import pickle
import matplotlib.pyplot as plt

import re, os
for dirname, _, filenames in os.walk('data/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('data/dataset/suspicious-tweets.csv')

df.groupby('label').describe()

In [ ]:
# Count the number of characters in each message
df['length'] = df['message'].apply(len)
df.head()

In [ ]:
# Use for pre-test only 11 number of elements!
df_labels = df['label']
df_labels.head()

In [ ]:
# Only for hinstogram
count_Class = pd.value_counts(df.label, sort = True)

# Data
labels = '[0]suspicious', '[1]non-suspicious'
sizes = [count_Class[0], count_Class[1]]
colors = ['red', 'skyblue']
explode = (0.1, 0.1)


plt.pie(sizes, explode = explode, labels = labels, colors = colors,
        autopct = '%1.1f%%', shadow = True, startangle = 90)
plt.axis('equal')
plt.show()

In [ ]:
# Preprocess text and clean string
def preprocess_text(sen):
    sentence = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)',' ',sen) # Removing html tags
    sentence = re.sub('[^a-zA-Z]', ' ', sentence) # Remove punctuations and numbers
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence) # Single character removal
    sentence = re.sub(r'\s+', ' ', sentence) # Removing multiple spaces
    sentence = sentence.replace("ain't", "am not").replace("aren't", "are not")
    sentence = ' '.join(text.lower() for text in sentence.split(' ')) # Lowering cases
    sw = stopwords.words('english')
    sentence = ' '.join(text for text in sentence.split() if text not in sw) #removing stopwords
    #sentence = ' '.join(text.lemmatize() for text in sentence.split()) #lemmatization
    return sentence

#### Lib
--------------
The **Porter stemming** algorithm (or 'Porter stemmer') is a process for removing the commoner morphological and inflexional endings from words in English. Its main use is as part of a term normalisation process that is usually done when setting up Information Retrieval systems.

In [ ]:
# Save in message, clean text
df['message'] = df.message.apply(preprocess_text)
# Tokenization each text
df['message'] = df['message'].apply(nltk.word_tokenize)

stemmer = PorterStemmer()
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
# is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
# -> ['upset', 'updat', 'facebook', 'text', 'might', 'cri', 'result', 'school', 'today', 'blah']
show = df['message']
show.head()

Lib
-------------------
**CountVectorizer**
Convert a collection of text documents to a matrix of token counts.

This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.

**TfidfTransformer**
Transform a count matrix to a normalized tf or tf-idf representation.

Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

In [ ]:
df['message'] = df['message'].apply(lambda x: ' '.join(x))

# Convert a collection of text documents to a matrix of token counts.
count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message']);

In [ ]:
transformer = TfidfTransformer().fit(counts)
counts = transformer.fit_transform(counts)

In [ ]:
# Separe dataset in traning and test(30%)
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.3, random_state=69)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# implementing naive bayes
NB = MultinomialNB().fit(X_train, y_train)

predicted = NB.predict(X_test)
print(np.mean(predicted == y_test))

In [ ]:
from sklearn.metrics import accuracy_score

# Confronto etichette corrette dei dati di test (y) con le risposte elaborate dal programma (predicted).
error = accuracy_score(y_test, predicted)
print(f"Accuracy: {error}")

In [ ]:
report = classification_report(y_test,predicted)
print(report)

In [ ]:
# Compute confusion matrix to evaluate the accuracy of a classification
print(confusion_matrix(y_test, predicted))

### SVM
----------
Le macchine vettoriali di supporto (SVM) sono un insieme di metodi di apprendimento supervisionato utilizzati per la classificazione, la regressione e il rilevamento di anomalie.

I vantaggi delle macchine a vettori di supporto sono
- Efficaci in spazi ad alta dimensionalità.
- Sono ancora efficaci nei casi in cui il numero di dimensioni è maggiore del numero di campioni.
- Utilizza un sottoinsieme di punti di addestramento nella funzione di decisione (chiamati vettori di supporto), quindi è anche efficiente dal punto di vista della memoria.
- Versatile: è possibile specificare diverse funzioni kernel per la funzione di decisione. Vengono forniti kernel comuni, ma è anche possibile specificare kernel personalizzati.

Gli svantaggi delle macchine vettoriali di supporto sono i seguenti:
- Se il numero di caratteristiche è molto maggiore del numero di campioni, è fondamentale evitare l'over-fitting nella scelta delle funzioni Kernel e del termine di regolarizzazione.
- Le SVM non forniscono direttamente stime di probabilità, che vengono calcolate utilizzando una costosa convalida incrociata a cinque volte (vedere Punteggi e probabilità, più avanti).
Le macchine vettoriali di supporto in scikit-learn supportano come input vettori campione sia densi (numpy.ndarray e convertibili in numpy.asarray) che radi (qualsiasi scipy.sparse). Tuttavia, per utilizzare una SVM per fare previsioni su dati sparsi, è necessario che sia stata adattata a tali dati. Per ottenere prestazioni ottimali, utilizzare numpy.ndarray ordinati in C (densi) o scipy.sparse.csr_matrix (radi) con dtype=float64.

In [ ]:
# implementing SVM
sv = SVC().fit(X_train, y_train)

predicted = sv.predict(X_test)
print(np.mean(predicted == y_test))

In [ ]:
print(classification_report(y_test,predicted))

### Confusion matrix
------------
Compute confusion matrix to evaluate the accuracy of a classification.

By definition a confusion matrix `C` is such that `C_{i, j}` is equal to the number of observations known to be in group `i` and predicted to be in group `j`.

Thus in binary classification, the count of true negatives is `C_{0,0}`, false negatives is `C_{1,0}`, true positives is `C_{1,1}` and false positives is `C_{0,1}`.

##### Return:
`C `: ndarray of shape (n_classes, n_classes)
    Confusion matrix whose `i-th` row and `j-th` column entry indicates the number of samples with true label being `i-th` class and predicted label being `j-th` class.

In [ ]:
print(confusion_matrix(y_test, predicted))

### Decision tree classifier
------------
Decision tree builds classification or regression models in the form of a tree structure. It breaks down a dataset into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes.

In [ ]:
# implementing disession tree
dt = DecisionTreeClassifier().fit(X_train, y_train)

predicted = dt.predict(X_test)
print(np.mean(predicted == y_test))

In [ ]:
print(classification_report(y_test,predicted))

In [ ]:
print(confusion_matrix(y_test, predicted))

### An AdaBoost classifier
------------
An AdaBoost classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.

This class implements the algorithm known as AdaBoost-SAMME.

In [ ]:
# implementing adabost
ab = AdaBoostClassifier().fit(X_train, y_train)

In [ ]:
predicted = ab.predict(X_test)

print(np.mean(predicted == y_test))

In [ ]:
print(classification_report(y_test,predicted))

In [ ]:
print(confusion_matrix(y_test, predicted))

##### Lib
-------------
**Transform** documents to document-term matrix.
Extract token counts out of raw text documents using the vocabulary fitted with fit or the one provided to the constructor.

**Predict** class or regression value for X.
For a classification model, the predicted class for each sample in X is returned. For a regression model, the predicted value based on X is returned.

In [ ]:
# converting content to lower case
pred = (df['message'].str.lower())
# printing predictions made by model
print("prediction: {}". format(dt.predict(count_vect.transform(pred.values.astype('U')))))
# saving predictions in a variable
my_pred = dt.predict(count_vect.transform(pred.values.astype('U')))

# saving predicted labels in .csv file
df['autotag'] = my_pred
df.to_csv('data/dataset/result.csv',index = False)

In [ ]:
# getting the number of -ve , +ve  predictions 
positive = 0
negative = 0

for v in my_pred:
 if (v == 1):
  positive += 1
 elif (v == 0):
  negative += 1

In [ ]:
# function for quality measurement (can be set to required parameters)
def quality(pos,total):
    if((pos*100)/total >= 0 and (pos*100)/total <=24 ):
        print('Quality: Very Negative')
    elif((pos*100)/total >= 25 and (pos*100)/total <=49 ):
        print('Quality: Negative')
    elif((pos*100)/total >= 50 and (pos*100)/total <=74 ):
        print('Quality: Positive')
    elif((pos*100)/total >= 75 and (pos*100)/total <=100 ):
        print('Quality: Very Positive')

In [ ]:
# importing library to plot charts
import matplotlib.pyplot as plt 
  
# x-coordinates of left sides of bars  
left = [100, 200 ] 
  
# heights of bars 
height = [positive,negative] 
  
# labels for bars 
tick_label = ['Positive', 'Negative'] 
  
# plotting a bar chart 
plt.bar(left, height, tick_label = tick_label, width = 80, color = ['green','blue']) 
  
# naming the x-axis 
plt.xlabel('Sentiment') 
# naming the y-axis 
plt.ylabel('Reviews') 
# plot title 
plt.title('Sentiment Analysis') 
  
# function to show the plot 
plt.show() 
print ("Sentiment - Positive: "+str(positive)+", Negative: "+str(negative))
quality(positive,(positive+negative))

In [ ]:
# getting negativity 
negativeity = (negative*100)/(positive+negative)
positivity = (positive*100)/(positive+negative)
print("Negativeity: "+str(negativeity)+"%")
print("Positivity: "+str(positivity)+"%")

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(dt, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)